Universidade Federal de Alagoas

IC - Instituto de Computação

 

# Processamento de linguagem natural - 2020.1
**Professor**: Thales Vieira

**Alunos**: Hugo Tallys Martins Oliveira e Valério Nogueira Rodrigues Júnior


## 5ª lista de exercícios

---

# Pré-processamento dos dados

In [14]:
import re
import time
import umap
import nltk
import numpy
import pandas
import random

from bokeh.plotting import figure
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from bokeh.palettes import Category20
from gensim.models import KeyedVectors
from sklearn.decomposition import PCA
from IPython.display import HTML, display
from bokeh.io import output_notebook, show
from yellowbrick.cluster import KElbowVisualizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [15]:
output_notebook() # Necessário para visualizar os gráficos com bokeh

Loading BokehJS ...

In [16]:
nltk.download('stopwords'); nltk.download('rslp'); nltk.download('punkt');

[nltk_data] Downloading package stopwords to C:\Users\Valerio
[nltk_data]     Nogueira\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to C:\Users\Valerio
[nltk_data]     Nogueira\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Valerio
[nltk_data]     Nogueira\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
bbc_dataset_url = 'data/bbc.csv'
cnn_dataset_url = 'data/cnn.csv'

bbc = pandas.read_csv(bbc_dataset_url, sep='|')
bbc['label'] = 'BBC'
cnn = pandas.read_csv(cnn_dataset_url, sep='|')
cnn['label'] = 'CNN'

dataset = pandas.concat([bbc, cnn], ignore_index=True)
dataset = dataset.dropna(axis=0).reset_index(drop=True)

In [18]:
dataset.text = dataset.text.apply(lambda text: text.replace('\n', ' ')) # Remoção das quebras de linha
dataset.title = dataset.title.apply(lambda text: text.replace('\n', ' '))

In [19]:
def remove_boilerplate(text):
    boilerplate = ['Compartilhe este post com Email Facebook Messenger Messenger Twitter WhatsApp LinkedIn Copiar este link Estes são links externos e abrirão numa nova janela', 'Já assistiu aos nossos novos vídeos no YouTube? Inscreva-se no nosso canal!', 'Final de YouTube post  de BBC News Brasil Final de YouTube post 2 de BBC News Brasil Final de YouTube post 3 de BBC News Brasil']
    
    for b in boilerplate:
        text = text.replace(b, '')
    return text

dataset.text = dataset.text.apply(remove_boilerplate) # Remoção de fragmentos irrelevantes do texto que se repetem

In [20]:
def preprocess(text):
    text = re.sub(r'\w*\d\w*', '', text) # Remove todas as palavras que contém números
    text = re.sub(r'[^a-zA-ZáéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]', '', text.lower()) # Remove pontuação e converte para minúscula
    return re.sub(r'\s+', ' ', text) # Remove espaços repetidos

dataset['processed_text'] = dataset.text.apply(preprocess)
dataset['processed_title'] = dataset.title.apply(preprocess)

In [21]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def tokenize_remove_stopwords(text):
    tokenized_text = nltk.word_tokenize(text, language='portuguese')
    return " ".join([token for token in tokenized_text if token not in stopwords])

dataset.processed_text = dataset.processed_text.apply(tokenize_remove_stopwords) # Tokeniza o texto e remove stopwords
dataset.processed_title = dataset.processed_title.apply(tokenize_remove_stopwords) # Tokeniza o titulo e remove stopwords

In [22]:
dataset

,url,title,text,label,processed_text,processed_title
0,https://www.bbc.co.uk/portuguese/brasil-53020785,Coronavírus: pandemia pode jogar até 14 milhõe...,A turbulência econômica causada pela pandemi...,BBC,turbulência econômica causada pandemia novo co...,coronavírus pandemia pode jogar milhões brasil...
1,https://www.bbc.co.uk/portuguese/brasil-53027318,Coronavírus: como funcionam as duas vacinas co...,Cerca de 11 mil voluntários brasileiros vão ...,BBC,cerca mil voluntários brasileiros vão receber ...,coronavírus funcionam duas vacinas contra covi...
2,https://www.bbc.co.uk/portuguese/brasil-51713943,Coronavírus: Brasil passa o Reino Unido e se t...,*atualizada às 18h20 de 12 de junho de 2020 ...,BBC,atualizada s junho brasil totalizou nesta sext...,coronavírus brasil passa reino unido torna seg...
3,https://www.bbc.co.uk/portuguese/internacional...,Coronavírus na Índia: com lockdown 'insustentá...,"Quando, em 24 de março, o governo indiano in...",BBC,março governo indiano iniciou estrito isolamen...,coronavírus índia lockdown insustentável índia...
4,https://www.bbc.co.uk/portuguese/internacional...,2ª onda do coronavírus? Irã vê aumento acelera...,O Irã registrou um rápido aumento no número ...,BBC,irã registrou rápido aumento número casos covi...,onda coronavírus irã vê aumento acelerado após...
...,...,...,...,...,...,...
1611,https://www.cnnbrasil.com.br/saude/2020/02/27/...,Farmácias têm falta de máscaras após confirmaç...,Com a confirmação do primeiro caso de contamin...,CNN,confirmação primeiro caso contaminação novo co...,farmácias têm falta máscaras após confirmação ...
1612,https://www.cnnbrasil.com.br/business/2020/02/...,Ibovespa tem nova queda com mercado ainda preo...,Preocupações com a propagação do novo coronaví...,CNN,preocupações propagação novo coronavírus poten...,ibovespa nova queda mercado ainda preocupado c...
1613,https://www.cnnbrasil.com.br/internacional/202...,Japonesa testa positivo pela segunda vez para ...,TÓQUIO - Uma guia de ônibus turístico no Japão...,CNN,tóquio guia ônibus turístico japão apresentou ...,japonesa testa positivo segunda vez coronavírus
1614,https://www.cnnbrasil.com.br/saude/2020/02/26/...,Primeiro brasileiro com coronavírus tem sintom...,O primeiro brasileiro com diagnóstico confir...,CNN,primeiro brasileiro diagnóstico confirmado cor...,primeiro brasileiro coronavírus sintomas brand...


# Buscador de documentos - Word2vec

Desenvolver um buscador de documentos:

1. Escolha e aplique um modelo do tipo _word2vec_ a seus textos, compatível com o idioma escolhido (inglês ou português).
2. Escolha 5 palavras de consulta que não estão em nenhum dos textos. Para cada palavra de consulta, encontre as 3 palavras __de seu conjunto de textos__ mais parecidas com cada uma das palavras de consulta e exiba os documentos onde estas palavras aparecem.
3. Usando as mesmas palavras do item acima, recupere os 3 documentos cujo _word vector_ médio é mais próximo de cada palavra de consulta.
4. Realize o procedimento acima usando três modelos com dimensão distinta.

Primeiro vamos determinar o vocabulário do nosso dataset (__corpo do documento__ e __título do documento__):

In [23]:
def get_vocab(texts):
    vocab = ' '.join([text for text in texts])
    return sorted(set(vocab.split(' ')))

processed_texts_vocab = get_vocab(dataset.processed_text.values)
processed_titles_vocab = get_vocab(dataset.processed_title.values)

Sorteando 5 palavras aleatoriamente do vocabulário de títulos:

In [24]:
search_words = random.sample(processed_titles_vocab, 5)
search_words

['bi', 'curado', 'empresas', 'levar', 'jogos']

Para determinar as palavras mais parecidas a cada palavra de busca selecionada vamos utilizar os seguintes modelos pré-treinados em português:

* CBOW 100
* SKIPGRAM 50
* SKIPGRAM 300

A descrição completa do modelo pré-treinado _word2vec_ pode ser encontrada no link:

> http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc

In [27]:
CBOW_100 = 'C:/Users/Valerio Nogueira/Documents/embeddings/cbow_s100.txt'
SKIP_GRAM_100 = 'C:/Users/Valerio Nogueira/Documents/embeddings/skip_s50.txt'
SKIP_GRAM_300 = 'C:/Users/Valerio Nogueira/Documents/embeddings/skip_s300.txt'

MODEL_LIST = [('CBOW_100', CBOW_100), ('SKIP_GRAM_100', SKIP_GRAM_100), ('SKIP_GRAM_300', SKIP_GRAM_300)]

Encontrando lista de palavras mais similares para cada modelo:

In [28]:
def get_most_similar_words(search_word, n):
    most_similar = []
    for word in processed_texts_vocab:
        try:
            similarity = embedding.similarity(word, search_word)
        except:
            similarity = -1
        most_similar.append(similarity)
    most_similar = numpy.argsort(most_similar)[-(n+1):-1]
    most_similar = most_similar[::-1]
    return [processed_texts_vocab[i] for i in most_similar]

most_similar_words = {}
for model_name, model_path in MODEL_LIST:
    print('\nCarregando o modelo %s' % model_name)
    embedding = KeyedVectors.load_word2vec_format(model_path)
    print('Calculando palavras mais similares')
    most_similar_words[model_name] = []
    for search_word in search_words:
        sim_search_word = get_most_similar_words(search_word, 3)
        print(f'\t{search_word} -> {str(sim_search_word)}')
        most_similar_words[model_name].append({'search_word': search_word, 'similar_words': sim_search_word})


Carregando o modelo CBOW_100
Calculando palavras mais similares
	bi -> ['th', 'tri', 'g']
	curado -> ['curada', 'recuperado', 'enganado']
	empresas -> ['montadoras', 'companhias', 'distribuidoras']
	levar -> ['trazer', 'entregar', 'devolver']
	jogos -> ['torneios', 'campeonatos', 'estádios']

Carregando o modelo SKIP_GRAM_100
Calculando palavras mais similares
	bi -> ['esp', 'r', 'aa']
	curado -> ['curada', 'doente', 'pegado']
	empresas -> ['companhias', 'cooperativas', 'seguradoras']
	levar -> ['trazer', 'estender', 'acompanhar']
	jogos -> ['torneios', 'paralímpicos', 'olímpicos']

Carregando o modelo SKIP_GRAM_300
Calculando palavras mais similares
	bi -> ['u', 'f', 'trilhão']
	curado -> ['curada', 'desidratado', 'comido']
	empresas -> ['multinacionais', 'seguradoras', 'companhias']
	levar -> ['trazer', 'conduzir', 'ir']
	jogos -> ['paralímpicos', 'olímpicos', 'verão']


Dado o __modelo__ e a __palavra de busca__ recuperamos alguns documentos que cotenham as palavras mais semelhantes encontradas no item anterior:

In [ ]:
def get_word_list(model_name, search_word):
    for word_list in most_similar_words[model_name]:
        if search_word == word_list['search_word']:
            return word_list['similar_words']

def fetch_documents(word_list):
    dataframes = []
    for word in word_list:
        df = dataset[dataset.processed_text.apply(lambda text: word in text.split(' '))]
        dataframes.append(df[['url', 'title']])
    return pandas.concat(dataframes).drop_duplicates().reset_index(drop=True)        

for model_name, model_path in MODEL_LIST:
    for search_word in search_words:
        word_list = get_word_list(model_name=model_name, search_word=search_word)
        search_documents = fetch_documents(word_list)
        display(HTML(f'<h3>Modelo [{model_name}]<br><br>Palavra Busca [{search_word}]<br><br>Palavras Similares {word_list}</h3>'))
        display(HTML(search_documents.head().to_html()))

A rotina abaixo calcula os três documentos onde seu __vetor médio__ é mais semelhante ao vetor de cada palavra de busca fornecida:

In [ ]:
def calculate_mean_word2vec(text):
    vectors = []
    for word in text.split():
        try:
            vectors.append(word_vectors[word.strip()])
        except:
            pass
    return numpy.mean(vectors, axis=0)
    
def most_similar_rows(df, col_name, vector, top_n=3):
    df['cosine'] = df[col_name].apply(lambda a: numpy.dot(a, vector) / (numpy.linalg.norm(a) * numpy.linalg.norm(vector)))
    df = df.sort_values(by=['cosine'], ascending=False)
    return df[['url', 'title']][:top_n]


# Calcula o vetor médio de cada texto
dataset['mean_word_vector'] = dataset.text.apply(calculate_mean_word2vec)

for model_name, model_path in MODEL_LIST:
    embedding = KeyedVectors.load_word2vec_format(model_path)
    word_vectors = embedding.wv
    display(HTML('<h2>Modelo - %s</h2>' % model_name))
    for search_word in search_words:
        display(HTML(f'<h3>{search_word}</h3>'))
        display(HTML(most_similar_rows(dataset, 'mean_word_vector', word_vectors[search_word]).to_html()))

# Representação Doc2Vec

Vamos treinar um modelo de representação _doc2vec_ para os nossos documentos. Dado o conjunto de textos (lista `paragraphs`) vamos gerar a representação vetorial `doc_embedding`:

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
paragraphs = dataset.processed_text.values
paragraphs = [paragraph.split(' ') for paragraph in paragraphs]

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(paragraphs)]

In [ ]:
doc_embedding = Doc2Vec(documents, vector_size=300, window=10, min_count=2, workers=4)

In [ ]:
X = numpy.array([doc_embedding[i] for i, text in enumerate(dataset.processed_text.values)])

# Classificadores

Balanceando o dataset para conter 350 artigos _BBC_ e 350 artigos _CNN_ (700 artigos no total):

In [ ]:
X = X[:2*bbc.shape[0]]

In [ ]:
label2index = {
    'BBC': 0, 'CNN': 1
}
y = [label2index[l] for l in dataset.label.values[:700]]

Separando o conjunto de treinamento e teste:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

Treinando os classificadores:

### Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

Treinamento (TF-IFD):

In [ ]:
classifier = LogisticRegression().fit(X_train, y_train)

Resultado da predição (TF-IDF):

In [ ]:
y_pred = classifier.predict(X_test)

Matriz de confusão (TF-IDF):

In [ ]:
from matplotlib import pyplot
from sklearn.metrics import plot_confusion_matrix

In [ ]:
plot_confusion_matrix(classifier, X_test, y_test)
pyplot.show()

Métricas de validação (TF-IDF):

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
log_metrics = {
    'Modelo': 'Regressão Logísitca + TF-IDF',
    'Acurácia': accuracy_score(y_test, y_pred),
    'Precisão': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred)
}
log_metrics

### Naive Bayes

### Support Vector Machines (SVM)

Treinamento (TF-IFD):

In [ ]:
from sklearn.svm import SVC

In [ ]:
classifier = SVC().fit(X_train, y_train)

Resultado da predição (TF-IDF):

In [ ]:
y_pred = classifier.predict(X_test)

Matriz de confusão (TF-IDF):

In [ ]:
plot_confusion_matrix(classifier, X_test, y_test)
pyplot.show()

Métricas de validação (TF-IDF):

In [ ]:
svm_metrics = {
    'Modelo': 'SVM + TF-IDF',
    'Acurácia': accuracy_score(y_test, y_pred),
    'Precisão': precision_score(y_test, y_pred),
    'Recall': recall_score(y_test, y_pred),
    'F1 Score': f1_score(y_test, y_pred)
}
svm_metrics

Comparando os classificadores:

In [ ]:
pandas.DataFrame([log_metrics, svm_metrics])

# Agrupamento

Realizar um agrupamento dos dados através das seguintes etapas:

* Aplicar o algotimo __PCA__ preservando _95%_ da variância nos dados
* Aplicar o algoritmo ___k_-means__ nos dados projetados, utilizando o método _elbow_ para encontrar o valor de _k_ ótimo

Vamos realizar _stemming_ no texto pré-processado e vetorizar o resultado:

In [ ]:
X_proj = PCA(n_components=.95).fit_transform(X)

Dimensão dos pontos projetados: 

In [ ]:
X_proj.shape[1]

Plotando o gráfico da distorção:

In [ ]:
visualizer = KElbowVisualizer(KMeans(), k=(15, 30), metric='distortion')
visualizer.fit(X_proj)
visualizer.show() 

Observando o gráfico acima escolhemos $k=20$

In [ ]:
k = 20
kmeans = KMeans(n_clusters=k).fit(X_proj)

Projetando os pontos no espaço de visualização via __TSNE__:

In [ ]:
start_time = time.time()
X_tsne = TSNE(n_components=2).fit_transform(X_proj)
interval_time = time.time() - start_time

Tempo de execução:

In [ ]:
print('%s segundos' % interval_time)

Gráfico de dispersão dos pontos:

In [ ]:
color_palette = Category20[k]

In [ ]:
def scatter_plot(X, labels):
    scatter_plot = figure(plot_width=1000, plot_height=500)
    scatter_plot.circle(X[:, 0], X[:, 1], size=10, line_color=[color_palette[l] for l in labels], fill_color=[color_palette[l] for l in labels], fill_alpha=.8)
    show(scatter_plot)

scatter_plot(X_tsne, kmeans.labels_)

Projetando os pontos no espaço de visualização via __UMAP__:

In [ ]:
start_time = time.time()
X_umap = umap.UMAP().fit_transform(X_proj)
interval_time = time.time() - start_time

Tempo de execução:

In [ ]:
print('%s segundos' % interval_time)

In [ ]:
scatter_plot(X_umap, kmeans.labels_)